<a href="https://colab.research.google.com/github/Hasifa1/Thyroid-cancer-prediction/blob/main/TD_feature_extraction_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:

# Define paths to your image dataset for malign, benign, and normal cases
malignant_path = '/content/drive/MyDrive/dataset_thyroid/malign'
benign_path = '/content/drive/MyDrive/dataset_thyroid/benign'
normal_path = '/content/drive/MyDrive/dataset_thyroid/normal'

In [ ]:

# Create data generators
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


In [ ]:

# Load and split the data
train_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset_thyroid',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',  # Make sure to use categorical mode for one-hot encoding
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset_thyroid',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 1287 images belonging to 3 classes.
Found 320 images belonging to 3 classes.


In [ ]:

# Extract features using a pre-trained model (e.g., VGG16)
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional base
base_model.trainable = False

# Create logistic regression model on top
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 classes: Malignant, Benign, Normal
])


58889256/58889256 [==============================] - 4s 0us/step


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(train_generator, epochs=10, validation_data=validation_generator)


Epoch 1/10
41/41 [==============================] - 1140s 28s/step - loss: 0.9886 - accuracy: 0.4654 - val_loss: 0.9181 - val_accuracy: 0.5125
Epoch 2/10
41/41 [==============================] - 9s 230ms/step - loss: 0.9256 - accuracy: 0.5136 - val_loss: 0.9095 - val_accuracy: 0.4062
Epoch 3/10
41/41 [==============================] - 10s 236ms/step - loss: 0.9126 - accuracy: 0.5198 - val_loss: 0.8884 - val_accuracy: 0.4156
Epoch 4/10
41/41 [==============================] - 10s 239ms/step - loss: 0.9009 - accuracy: 0.5260 - val_loss: 0.8671 - val_accuracy: 0.4187
Epoch 5/10
41/41 [==============================] - 9s 222ms/step - loss: 0.8954 - accuracy: 0.5291 - val_loss: 0.8703 - val_accuracy: 0.4125
Epoch 6/10
41/41 [==============================] - 9s 227ms/step - loss: 0.8927 - accuracy: 0.5245 - val_loss: 0.8482 - val_accuracy: 0.4750
Epoch 7/10
41/41 [==============================] - 10s 231ms/step - loss: 0.8860 - accuracy: 0.5338 - val_loss: 0.8474 - val_accuracy: 0.4406
Ep

In [ ]:
# Extract features for logistic regression
train_features = model.predict(train_generator)
validation_features = model.predict(validation_generator)


10/10 [==============================] - 2s 174ms/step


In [ ]:
# Flatten the features
train_features_flat = train_features.reshape((train_features.shape[0], -1))
validation_features_flat = validation_features.reshape((validation_features.shape[0], -1))

In [ ]:

# Create logistic regression model
logistic_regression_model = LogisticRegression(max_iter=1000)

# Train the logistic regression model
logistic_regression_model.fit(train_features_flat, train_generator.classes)

# Evaluate the logistic regression model
logistic_regression_predictions = logistic_regression_model.predict(validation_features_flat)

# Print classification report and confusion matrix
print(classification_report(validation_generator.classes, logistic_regression_predictions))
print(confusion_matrix(validation_generator.classes, logistic_regression_predictions))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       126
           1       0.50      1.00      0.67       160
           2       0.00      0.00      0.00        34

    accuracy                           0.50       320
   macro avg       0.17      0.33      0.22       320
weighted avg       0.25      0.50      0.33       320

[[  0 126   0]
 [  0 160   0]
 [  0  34   0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
accuracy = accuracy_score(validation_generator.classes, logistic_regression_predictions)
# Print classification report, confusion matrix, and accuracy
print(classification_report(validation_generator.classes, logistic_regression_predictions))
print(confusion_matrix(validation_generator.classes, logistic_regression_predictions))
print(f'Accuracy: {accuracy * 100:.2f}%')

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       126
           1       0.50      1.00      0.67       160
           2       0.00      0.00      0.00        34

    accuracy                           0.50       320
   macro avg       0.17      0.33      0.22       320
weighted avg       0.25      0.50      0.33       320

[[  0 126   0]
 [  0 160   0]
 [  0  34   0]]
Accuracy: 50.00%


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
